In [18]:
pip install -r requirements.txt -q

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [235]:
import json
import time
import boto3
import re
import pandas as pd
from io import StringIO
# import sentencepiece
import pandas as pd
from anthropic import Anthropic
CLAUDE = Anthropic()
import multiprocessing
import subprocess
import shutil
import os
import codecs
import uuid
REDSHIFT=boto3.client('redshift-data')
S3=boto3.client('s3')
from botocore.config import Config
config = Config(
    read_timeout=600,
    retries = dict(
        max_attempts = 5
    )
)
from botocore.exceptions import ClientError
bedrock_runtime=boto3.client(service_name='bedrock-runtime',region_name='us-east-1',config=config)

## REDSHIFT

Utility functions

In [236]:
def bedrock_streemer(response):
    stream = response.get('body')
    answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1
                if "amazon-bedrock-invocationMetrics" in chunk_obj:
                    input_tokens= chunk_obj['amazon-bedrock-invocationMetrics']['inputTokenCount']
                    output_tokens=chunk_obj['amazon-bedrock-invocationMetrics']['outputTokenCount']
                    print(f"\nInput Tokens: {input_tokens}\nOutput Tokens: {output_tokens}")
    return answer,input_tokens, output_tokens

def bedrock_claude_(chat_history,params, prompt):
    content=[{
        "type": "text",
        "text": prompt
            }]
    chat_history.append({"role": "user",
            "content": content})
    prompt = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": params["token"],
        "temperature": params["temp"],
        "system":params["system_message"],
        "messages": chat_history
    }
    answer = ""
    prompt = json.dumps(prompt)
    # print(chat_history)
    response = bedrock_runtime.invoke_model_with_response_stream(body=prompt, modelId=params["sql_model"], accept="application/json", contentType="application/json")
    answer,input_tokens,output_tokens=bedrock_streemer(response) 
    return answer, input_tokens, output_tokens

def _invoke_bedrock_with_retries(chat_history, params, question):
    max_retries = 5
    backoff_base = 2
    max_backoff = 3  # Maximum backoff time in seconds
    retries = 0

    while True:
        try:
            response,input_tokens,output_tokens= bedrock_claude_(chat_history, params, question)
            return response,input_tokens,output_tokens
        except ClientError as e:
            if e.response['Error']['Code'] == 'ThrottlingException':
                if retries < max_retries:
                    # Throttling, exponential backoff
                    sleep_time = min(max_backoff, backoff_base ** retries + random.uniform(0, 1))
                    time.sleep(sleep_time)
                    retries += 1
                else:
                    raise e
            elif e.response['Error']['Code'] == 'ModelStreamErrorException':
                if retries < max_retries:
                    # Throttling, exponential backoff
                    sleep_time = min(max_backoff, backoff_base ** retries + random.uniform(0, 1))
                    time.sleep(sleep_time)
                    retries += 1
                else:
                    raise e
            else:
                # Some other API error, rethrow
                raise
                

In [237]:
def query_llm(prompts,params):   
    """
    Function to prompt the model to generate sql statements from natural language
    """
    import boto3
    import json   
    answer, input_token, output_token=_invoke_bedrock_with_retries([], params, prompts)        
    return answer



def qna_llm(prompts,params):
    """
    Function to prompt the model to generate natural language answers from sql results
    """
    import json 
    answer, input_token, output_token=_invoke_bedrock_with_retries([], params, prompts)
    return answer

In [238]:
def chunk_csv_rows(csv_rows, max_token_per_chunk):
    """
    Chunk CSV rows based on the maximum token count per chunk.
    Args:
        csv_rows (list): List of CSV rows.
        max_token_per_chunk (int, optional): Maximum token count per chunk.
    Returns:
        list: List of chunks containing CSV rows.
    Raises:
        ValueError: If a single CSV row exceeds the specified max_token_per_chunk.
    """
    header = csv_rows[0]  # Assuming the first row is the header
    csv_rows = csv_rows[1:]  # Remove the header from the list
    current_chunk = []
    current_token_count = 0
    chunks = []
    header_token=len(mixtral_counter("mistralai/Mixtral-8x7B-v0.1").encode(header))
    for row in csv_rows:
        token = len(mixtral_counter("mistralai/Mixtral-8x7B-v0.1").encode(row))
        if current_token_count + token+header_token <= max_token_per_chunk:
            current_chunk.append(row)
            current_token_count += token
        else:
            if not current_chunk:
                raise ValueError("A single CSV row exceeds the specified max_token_per_chunk.")
            header_and_chunk=[header]+current_chunk
            chunks.append("\n".join([x for x in header_and_chunk]))
            current_chunk = [row]
            current_token_count = token

    if current_chunk:
        last_chunk_and_header=[header]+current_chunk
        chunks.append("\n".join([x for x in last_chunk_and_header]))
    return chunks

In [239]:
def get_tables_redshift(identifier, database,  schema,serverless,db_user=None,):
    """
    Get a list of table names in a specified schema from an Amazon Redshift cluster.
    Args:
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database containing the tables.
        db_user (str): The username used to authenticate with the Redshift cluster.
        schema (str): The schema pattern to filter tables.
    Returns:
        list: A list of table names in the specified schema.
    """
    if serverless:
        tables_ls = REDSHIFT.list_tables(
       WorkgroupName=identifier,
        Database=database,
        SchemaPattern=schema
        )
    else:
        tables_ls = REDSHIFT.list_tables(
        ClusterIdentifier=identifier,
        Database=database,
        DbUser=db_user,
        SchemaPattern=schema
        )
    return [x['name'] for x in  tables_ls['Tables']]

In [240]:
def get_db_redshift(identifier, database,serverless, db_user=None, ):
    """
    Get a list of databases from an Amazon Redshift cluster.
    Args:
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database containing the tables.
        db_user (str): The username used to authenticate with the Redshift cluster.
    Returns:
        list: A list of databases in the Redshift cluster.
    """
    if serverless:
        db_ls = REDSHIFT.list_databases(
        WorkgroupName=identifier,
        Database=database,
        )    
    else:
        db_ls = REDSHIFT.list_databases(
        ClusterIdentifier=identifier,
        Database=database,
        DbUser=db_user
        )
    return db_ls['Databases']

In [241]:
def get_schema_redshift(identifier, database, serverless, db_user=None,):
    """
    Get a list of schemas from an Amazon Redshift cluster.
    Args:
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database containing the schemas.
        db_user (str): The username used to authenticate with the Redshift cluster.
    Returns:
        list: A list of schemas in the Redshift cluster.
    """
    if serverless:
        schema_ls = REDSHIFT.list_schemas(
        WorkgroupName=identifier,
        Database=database,

        )
    else:        
        schema_ls = REDSHIFT.list_schemas(
        ClusterIdentifier=identifier,
        Database=database,
        DbUser=db_user
        )
    return schema_ls['Schemas']

In [242]:
def execute_query_with_pagination( sql_query, identifier, database,  serverless, db_user=None,):
    """
    Execute multiple SQL queries in Amazon Redshift with pagination support.
    Args:
        sql_query1 (str): The first SQL query to execute.
        sql_query2 (str): The second SQL query to execute.
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database.
        db_user (str): The username used to authenticate with the Redshift cluster.
    Returns:
        list: A list of results from executing the SQL queries.
    """
    results_list=[]
    if serverless:
        response_b = REDSHIFT.batch_execute_statement(
            WorkgroupName=identifier,
            Database=database,
        
            Sqls=sql_query
        ) 
    else:
        response_b = REDSHIFT.batch_execute_statement(
            ClusterIdentifier=identifier,
            Database=database,
            DbUser=db_user,
            Sqls=sql_query
        )   
    describe_b=REDSHIFT.describe_statement(
         Id=response_b['Id'],
    )       
    status=describe_b['Status']
    print(status)
    while status != "FINISHED":
        time.sleep(1)
        describe_b=REDSHIFT.describe_statement(
                         Id=response_b['Id'],
                    ) 
        status=describe_b['Status']
        if status=="FAILED":
            raise("Execution Failed")
        
        print(status)
    max_attempts = 5 
    attempts = 0
    while attempts < max_attempts:
        try:
            for ids in describe_b['SubStatements']:
                result_b = REDSHIFT.get_statement_result(Id=ids['Id'])                
                results_list.append(get_redshift_table_result(result_b))
            break
        except REDSHIFT.exceptions.ResourceNotFoundException as e:
            attempts += 1
            time.sleep(2)
    return results_list

In [243]:
def get_redshift_table_result(response):
    """
    Extracts result data from a Redshift query response and returns it as a CSV string.
    Args:
        response (dict): The response object from a Redshift query.
    Returns:
        str: A CSV string containing the result data.
    """
    columns = [c['name'] for c in response['ColumnMetadata']] 
    data = []
    for r in response['Records']:
        row = []
        for col in r:
            row.append(list(col.values())[0])  
        data.append(row)
    df = pd.DataFrame(data, columns=columns)    
    return df.to_csv(index=False)

In [244]:
def execute_query_redshyft(sql_query, identifier, database, serverless,db_user=None):
    """
    Execute a SQL query on an Amazon Redshift cluster.
    Args:
        sql_query (str): The SQL query to execute.
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database.
        db_user (str): The username used to authenticate with the Redshift cluster.
    Returns:
        dict: The response object from executing the SQL query.
    """
    if serverless:
        response = REDSHIFT.execute_statement(
        WorkgroupName=identifier,
            Database=database,

            Sql=sql_query
        )
    else:        
        response = REDSHIFT.execute_statement(
            ClusterIdentifier=identifier,
            Database=database,
            DbUser=db_user,
            Sql=sql_query
        )
    return response

In [245]:
def single_execute_query(params,sql_query, identifier, database, question,serverless,db_user=None):
    """
    Execute a single SQL query on an Amazon Redshift cluster and process the result.

    Args:
        sql_query (str): The SQL query to execute.
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database.
        db_user (str): The username used to authenticate with the Redshift cluster.
        question (str): A descriptive label or question associated with the query.

    Returns:
        pandas.DataFrame: DataFrame containing the processed result of the SQL query.

    """

    response = execute_query_redshyft(sql_query, identifier, database,serverless, db_user)
    df=redshyft_querys(sql_query,response,question,params,identifier, 
                       database,                     
                       question,
                         db_user,)    
    return df

In [246]:
def llm_debugga(question, statement, error, params): 
    """
    Generate debugging guidance and expected SQL correction for a PostgreSQL error.
    Args:
        question (str): The user's question or intent.
        statement (str): The SQL statement that caused the error.
        error (str): The error message encountered.
        params (dict): Additional parameters including schema, sample data, and length.
    Returns:
        str: Formatted debugging guidance and expected SQL correction.
    """
 
    model="claude"
    with open(f"prompt/{params['prompt-type']}/{model}-debugger.txt","r") as f:
        prompts=f.read()
    values = {
    "error":error,
    "sql":statement,
    "schema": params['schema'],
    "sample": params['sample'],
    "question":params['prompt']
    }
    prompts=prompts.format(**values)
    answer=query_llm(prompts,params)
    return answer

In [247]:
def redshyft_querys(q_s,response,prompt,params,identifier, database, question,db_user=None,): 
    """
    Execute a Redshift query, handle errors, debug SQL, and return the result.

    Args:
        q_s (str): The SQL statement to execute or debug.
        response (dict): The response object from executing the SQL statement.
        prompt (str): The user's question or intent.
        params (dict): Additional parameters including schema, sample data, and length.
        identifier (str): The identifier of the Redshift cluster.
        database (str): The name of the database.
        db_user (str): The username used to authenticate with the Redshift cluster.
        question (str): A descriptive label or question associated with the query.

    Returns:
        pandas.DataFrame or str: DataFrame containing the query result, or debugging failure message with no result.

    """
    max_execution=5
    debug_count=max_execution
    alert=False
    try:
        statement_result = REDSHIFT.get_statement_result(
            Id=response['Id'],
        )
    except REDSHIFT.exceptions.ResourceNotFoundException as err:  
        # print(err)
        describe_statement=REDSHIFT.describe_statement(
             Id=response['Id'],
        )
        query_state=describe_statement['Status']  
        while query_state in ['SUBMITTED','PICKED','STARTED']:
            # print(query_state)
            time.sleep(1)
            describe_statement=REDSHIFT.describe_statement(
                 Id=response['Id'],
            )
            query_state=describe_statement['Status']
        while (max_execution > 0 and query_state == "FAILED"):
            max_execution = max_execution - 1
            print(f"\nDEBUG TRIAL {max_execution}")
            bad_sql=describe_statement['QueryString']
            print(f"\nBAD SQL:\n{bad_sql}")                
            error=describe_statement['Error']
            print(f"\nERROR:{error}")
            print("\nDEBUGGIN...")
            cql=llm_debugga(prompt, bad_sql, error, params)            
            idx1 = cql.index('<sql>')
            idx2 = cql.index('</sql>')
            q_s=cql[idx1 + len('<sql>') + 1: idx2]
            print(f"\nDEBUGGED SQL\n {q_s}")
            ### Guardrails to prevent the LLM from altering tables
            if any(keyword in q_s for keyword in ["CREATE", "DROP", "ALTER","INSERT","UPDATE","TRUNCATE","DELETE","MERGE","REPLACE","UPSERT"]):
                alert="I AM NOT PERMITTED TO MODIFY THIS TABLE, CONTACT ADMIN."
                print(alert)
                alert=True
                break
            else:
                response = execute_query_redshyft(q_s, identifier, database,params['serverless'],db_user)
                describe_statement=REDSHIFT.describe_statement(
                                     Id=response['Id'],
                                )
                query_state=describe_statement['Status']
                while query_state in ['SUBMITTED','PICKED','STARTED']:
                    time.sleep(2)            
                    describe_statement=REDSHIFT.describe_statement(
                                     Id=response['Id'],
                                )
                    query_state=describe_statement['Status']
                if query_state == "FINISHED":                
                    break 
        
        if max_execution == 0 and query_state == "FAILED":
            print(f"DEBUGGING FAILED IN {str(debug_count)} ATTEMPTS")
        elif alert:
            pass
        else:           
            max_attempts = 5
            attempts = 0
            while attempts < max_attempts:
                try:
                    time.sleep(1)
                    statement_result = REDSHIFT.get_statement_result(
                        Id=response['Id']
                    )
                    break

                except REDSHIFT.exceptions.ResourceNotFoundException as e:
                    attempts += 1
                    time.sleep(5)
    if max_execution == 0 and query_state == "FAILED":
        df=f"DEBUGGING FAILED IN {str(debug_count)} ATTEMPTS. NO RESULT AVAILABLE"
    elif alert:
        df="I AM NOT PERMITTED TO MODIFY THIS TABLE, CONTACT ADMIN."     
    else:
        df=get_redshift_table_result(statement_result)
    return df, q_s

In [248]:
def redshift_qna(params):
    """
    Execute a Q&A process for generating SQL queries based on user questions.
    Args:
        params (dict): A dictionary containing parameters including table name, database name, prompt, etc.
    Returns:
        tuple: A tuple containing the response, generated SQL statement, and query output.
    """
    sql1=f"SELECT table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type FROM information_schema.columns WHERE table_schema='{params['db-schema']}'"
    sql2=[]
    for table in params['tables']:
        sql2.append(f"SELECT * from {params['database']}.{params['db-schema']}.{table} LIMIT 3")
    sqls=[sql1]+sql2
    
    question=params['prompt']
    results=execute_query_with_pagination(sqls, IDENTIFIER, params['database'],params['serverless'], DB_USER)    
    # print(results)
    col_names=results[0].split('\n')[0]
    
    observations="\n".join(sorted(results[0].split('\n')[1:])).strip()

    params['schema']=f"{col_names}\n{observations}"
    
    params['sample']=''
    for ids,examples in enumerate(results[1:]):
        params['sample']+=f"<{params['tables'][ids]}_table>\n{examples}</{params['tables'][ids]}_table>\n"

    model="claude"
    with open(f"prompt/{params['prompt-type']}/{model}-sql.txt","r") as f:
        prompts=f.read()
    values = {
    "schema": params['schema'],
    "sample": params['sample'],
    "question": question,
    }
    prompts=prompts.format(**values)
    # print(prompts)
    q_s=query_llm(prompts,params)
    sql_pattern = re.compile(r'<sql>(.*?)(?:</sql>|$)', re.DOTALL)           
    sql_match = re.search(sql_pattern, q_s)
    q_s = sql_match.group(1) 
    ### Guardrails to prevent the LLM from altering tables
    if any(keyword in q_s for keyword in ["CREATE", "DROP", "ALTER","INSERT","UPDATE","TRUNCATE","DELETE","MERGE","REPLACE","UPSERT"]):
        output="I AM NOT PERMITTED TO MODIFY THIS TABLE, CONTACT ADMIN."
        response="I AM NOT PERMITTED TO MODIFY THIS TABLE, CONTACT ADMIN."
    else:
        output, q_s=single_execute_query(params,q_s, IDENTIFIER, params['database'], question, params['serverless'],DB_USER) 
        # Handle results that exceed LLM token window length
        input_token=CLAUDE.count_tokens(output) if "claude" in params['text-model'].lower() else mistral_counter("mistralai/Mixtral-8x7B-v0.1").encode(output)
        if input_token>180000:    
            csv_rows=output.split('\n')
            chunk_rows=chunk_csv_rows(csv_rows, 50000)
            initial_summary=[]
            for chunk in chunk_rows:
                model="claude" 
                with open(f"prompt/{params['prompt-type']}/{model}-text-gen.txt","r") as f:
                    prompts=f.read()
                values = {   
                "sql":q_s,
                "csv": output,       
                "question":question,
                }
                prompts=prompts.format(**values)
                initial_summary.append(qna_llm(prompts,params))
            prompts = f'''You are a helpful and truthful assistant.
Here are multiple answer for a question on different subset of a tabular data:
#######
{initial_summary}
#######
Question: {question}
Based on the given question above, merege all answers provided in a coherent singular answer'''
            response=qna_llm(prompts,params)

        else:        
            model="claude" 
            with open(f"prompt/{params['prompt-type']}/{model}-text-gen.txt","r") as f:
                prompts=f.read()
            values = {   
            "sql":q_s,
            "csv": output,       
            "question":question,
            }
            prompts=prompts.format(**values)                
            response=qna_llm(prompts, params) 
    return response, q_s,output

#### Change parameters below to those of your Redshift resource
For Redshift provisioned capacity set `IDENTIFIER` to your cluster-identifier for your redshift cluster and `DB_USER` to admin username and set `serverless` to False \
For redshidft serverless, set `IDENTIFIER` to your work-group name and set `serverless` to True

Change the **database_index** and **schema_index** to the specific database and schema where the tables are located

In [262]:
IDENTIFIER = 'redshift-cluster-llm'
DATABASE = 'dev'
serverless=False # Toggle this to True if using Redshift Serverless
DB_USER = 'faynxe' if not serverless else None


db=get_db_redshift(IDENTIFIER, DATABASE,serverless,DB_USER) 
print(f"here is the list of Databse in your account {db}")
# Select the database you want to query
database_index=1

schm=get_schema_redshift(IDENTIFIER, db[database_index],serverless,DB_USER)
print(f"here is the list of Schema {schm} in {db[database_index]} databse")
# Select Schema index databse
schema_index=-1

tables=get_tables_redshift(IDENTIFIER, db[database_index],schm[schema_index],serverless,DB_USER)
print(f"here is the list of tables {tables} in {db[database_index]} databse and {schm[schema_index]} schema")

tables=[x for x in tables if "pkey" not in x] #Remove any non-table name from list of returned tables


here is the list of Databse in your account ['awsdatacatalog', 'dev']
here is the list of Schema ['catalog_history', 'information_schema', 'pg_auto_copy', 'pg_automv', 'pg_catalog', 'pg_internal', 'pg_s3', 'public'] in dev databse
here is the list of tables ['holdings_pkey', 'cars', 'category', 'date', 'event', 'holdings', 'listing', 'sales', 'users', 'venue'] in dev databse and public schema


You can decide the number of tables to be loaded into the LLM prompt for query. For each table selected, the schema definition and sample rows are loaded into the LLM prompt.
For single table query, you can pass the single table name in the params definition below for a given Database Schema.
For query that requires insight into multiple tables, pass the list of tables in the params definition below.

In [250]:
# Pass your question
question="What are the list of bond holdings?"

params={'sql-token':700,'token':500,'tables':tables, # Tables must be in list format
        'db-schema':schm, # Redshift Database SchemaName,
        "database":db,
        'temp':0.5,
        "serverless": serverless, 
        'text-model':'anthropic.claude-3-sonnet-20240229-v1:0', # SQL to NL model
        "prompt-type":"redshift",
        "sql_model":"anthropic.claude-3-sonnet-20240229-v1:0",#"anthropic.claude-3-sonnet-20240229-v1:0""anthropic.claude-v2","anthropic.claude-3-haiku-20240307-v1:0"
        "prompt":question,
       "chat_history":[],
       "system_message":""}

In [ ]:
%%time
# Quesry the redshift table
ress=redshift_qna(params)

In [252]:
# Result and SQL Statement
print(f"Answer:\n{ress[0]}\n\nSQL:\n{ress[1]}")

Answer:
Based on the SQL result, the list of bond holdings includes:

1. U.S. Treasury Bond
2. Apple Inc. Corporate Bond
3. U.S. Treasury Note
4. Apple Inc. Corporate Bond
5. Amazon.com, Inc. Corporate Bond
6. Microsoft Corporation Corporate Bond
7. Amazon.com, Inc. Corporate Bond

The result shows the security_name, ticker_symbol, quantity, market_value, coupon_rate, maturity_date, and credit_rating for each bond holding. The security_type filter in the SQL query ensures that only bonds are included in the result.

SQL:

SELECT security_name, ticker_symbol, quantity, market_value, coupon_rate, maturity_date, credit_rating
FROM dev.public.holdings
WHERE security_type = 'Bond';



In [253]:
# SQL Query Result
df=pd.read_csv(StringIO(ress[2]))
df

,security_name,ticker_symbol,quantity,market_value,coupon_rate,maturity_date,credit_rating
0,U.S. Treasury Bond,True,1000.0,95000.0,2.75,2028-05-15,AAA
1,Apple Inc. Corporate Bond,True,500.0,50000.0,3.25,2026-09-01,AA+
2,U.S. Treasury Note,True,750.0,72000.0,1.63,2025-02-15,AAA
3,Apple Inc. Corporate Bond,True,400.0,40000.0,2.85,2027-01-15,AA+
4,"Amazon.com, Inc. Corporate Bond",True,300.0,30000.0,3.38,2031-05-01,AA
5,Microsoft Corporation Corporate Bond,True,800.0,80000.0,3.00,2031-07-01,AAA
6,"Amazon.com, Inc. Corporate Bond",True,200.0,20000.0,3.15,2032-03-01,AA


In [279]:
prompt={
  "prompt": "hello",
  "max_tokens_to_sample": 70,
  "temperature": 0.1,
  # "top_k": 250,
  # "top_p":params['top_p'],  
     # "stop_sequences": []
}

# content=[{
#         "type": "text",
#         "text": prompt
#             }]


A solar eclipse occurs when the moon passes between the sun and the Earth, temporarily blocking the sun's rays and casting a shadow on parts of Earth. There are a few key things to know about solar eclipses:

- Total Solar Eclipse: This is when the moon's apparent size in the sky completely covers the sun, revealing the sun's outer atmosphere called the corona. The moon casts its umbra (darkest part of the shadow) on Earth's surface along a narrow path.


In [277]:
for stream in output['body']:
    print(stream)

In [271]:
response_body

{'id': 'msg_01D7r6AwfZgxCLP3yj7bNCAK',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text', 'text': 'Hello! How can I assist you today?'}],
 'model': 'claude-3-sonnet-28k-20240229',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 8, 'output_tokens': 12}}

In [ ]:
answer = ""
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if  chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if "delta" in chunk_obj:                    
                    delta = chunk_obj['delta']
                    if "text" in delta:
                        text=delta['text'] 
                        print(text, end="")
                        answer+=str(text)       
                        i+=1